In [79]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources\charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [80]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [81]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [82]:
# Look at APPLICATION_TYPE value counts for binning
Ap_type_counts = application_df.APPLICATION_TYPE.value_counts()
Ap_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [83]:
# Determine which values to replace if counts are less than ...?
replace_application = list(Ap_type_counts[Ap_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [84]:
# Look at CLASSIFICATION value counts for binning
class_type_counts = application_df.CLASSIFICATION.value_counts()
class_type_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [85]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_type_counts[class_type_counts < 1882].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [86]:
# Generate our categorical variable lists
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [87]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\jbrad\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [88]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\jbrad\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [89]:
application_df = application_df.drop(["STATUS", "SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y", "ASK_AMT"], axis=1)

application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)

In [91]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [103]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 70
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 30



nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
    )

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 90)                3600      
                                                                 
 dense_72 (Dense)            (None, 70)                6370      
                                                                 
 dense_73 (Dense)            (None, 50)                3550      
                                                                 
 dense_74 (Dense)            (None, 30)                1530      
                                                                 
 dense_75 (Dense)            (None, 1)                 31        
                                                                 
Total params: 15,081
Trainable params: 15,081
Non-trainable params: 0
_________________________________________________________________


In [104]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')


In [105]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150,callbacks =[cp_callback])

Epoch 1/150
756/804 [===========================>..] - ETA: 0s - loss: 0.5689 - accuracy: 0.7211
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 955us/step - loss: 0.5690 - accuracy: 0.7208
Epoch 2/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5541 - accuracy: 0.7323
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 940us/step - loss: 0.5548 - accuracy: 0.7320
Epoch 3/150
763/804 [===========================>..] - ETA: 0s - loss: 0.5528 - accuracy: 0.7325
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 947us/step - loss: 0.5524 - accuracy: 0.7326
Epoch 4/150
766/804 [===========================>..] - ETA: 0s - loss: 0.5504 - accuracy: 0.7327
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 956us/step - loss: 0.5504 - accuracy: 0.7327
Epoch 5/150
799/804 [===============

Epoch 35/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5378 - accuracy: 0.7400
Epoch 35: saving model to checkpoints\weights.35.hdf5
804/804 [==============================] - 1s 941us/step - loss: 0.5384 - accuracy: 0.7400
Epoch 36/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5383 - accuracy: 0.7406
Epoch 36: saving model to checkpoints\weights.36.hdf5
804/804 [==============================] - 1s 949us/step - loss: 0.5381 - accuracy: 0.7403
Epoch 37/150
772/804 [===========================>..] - ETA: 0s - loss: 0.5378 - accuracy: 0.7397
Epoch 37: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 1s 938us/step - loss: 0.5381 - accuracy: 0.7395
Epoch 38/150
774/804 [===========================>..] - ETA: 0s - loss: 0.5376 - accuracy: 0.7401
Epoch 38: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 945us/step - loss: 0.5378 - accuracy: 0.7398
Epoch 39/150
772/804 [======

Epoch 69/150
771/804 [===========================>..] - ETA: 0s - loss: 0.5354 - accuracy: 0.7404
Epoch 69: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 1s 940us/step - loss: 0.5342 - accuracy: 0.7413
Epoch 70/150
768/804 [===========================>..] - ETA: 0s - loss: 0.5328 - accuracy: 0.7413
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 954us/step - loss: 0.5338 - accuracy: 0.7408
Epoch 71/150
767/804 [===========================>..] - ETA: 0s - loss: 0.5339 - accuracy: 0.7409
Epoch 71: saving model to checkpoints\weights.71.hdf5
804/804 [==============================] - 1s 943us/step - loss: 0.5344 - accuracy: 0.7405
Epoch 72/150
773/804 [===========================>..] - ETA: 0s - loss: 0.5355 - accuracy: 0.7417
Epoch 72: saving model to checkpoints\weights.72.hdf5
804/804 [==============================] - 1s 946us/step - loss: 0.5360 - accuracy: 0.7417
Epoch 73/150
768/804 [======

Epoch 103/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5317 - accuracy: 0.7422
Epoch 103: saving model to checkpoints\weights.103.hdf5
804/804 [==============================] - 1s 941us/step - loss: 0.5323 - accuracy: 0.7414
Epoch 104/150
775/804 [===========================>..] - ETA: 0s - loss: 0.5332 - accuracy: 0.7417
Epoch 104: saving model to checkpoints\weights.104.hdf5
804/804 [==============================] - 1s 945us/step - loss: 0.5329 - accuracy: 0.7416
Epoch 105/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5326 - accuracy: 0.7416
Epoch 105: saving model to checkpoints\weights.105.hdf5
804/804 [==============================] - 1s 942us/step - loss: 0.5328 - accuracy: 0.7417
Epoch 106/150
771/804 [===========================>..] - ETA: 0s - loss: 0.5329 - accuracy: 0.7407
Epoch 106: saving model to checkpoints\weights.106.hdf5
804/804 [==============================] - 1s 939us/step - loss: 0.5323 - accuracy: 0.7413
Epoch 107/150
77

769/804 [===========================>..] - ETA: 0s - loss: 0.5333 - accuracy: 0.7417
Epoch 136: saving model to checkpoints\weights.136.hdf5
804/804 [==============================] - 1s 952us/step - loss: 0.5334 - accuracy: 0.7418
Epoch 137/150
772/804 [===========================>..] - ETA: 0s - loss: 0.5327 - accuracy: 0.7419
Epoch 137: saving model to checkpoints\weights.137.hdf5
804/804 [==============================] - 1s 939us/step - loss: 0.5328 - accuracy: 0.7420
Epoch 138/150
770/804 [===========================>..] - ETA: 0s - loss: 0.5348 - accuracy: 0.7417
Epoch 138: saving model to checkpoints\weights.138.hdf5
804/804 [==============================] - 1s 950us/step - loss: 0.5347 - accuracy: 0.7417
Epoch 139/150
769/804 [===========================>..] - ETA: 0s - loss: 0.5334 - accuracy: 0.7413
Epoch 139: saving model to checkpoints\weights.139.hdf5
804/804 [==============================] - 1s 941us/step - loss: 0.5333 - accuracy: 0.7413
Epoch 140/150
772/804 [=======

In [106]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5635 - accuracy: 0.7259 - 288ms/epoch - 1ms/step
Loss: 0.5634806752204895, Accuracy: 0.7259474992752075
